In [ ]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To shift lists
from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
import surprise as sp
from surprise.model_selection import cross_validate

# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from fuzzywuzzy import fuzz

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [ ]:
#Als Nächstes sollen alle Pfade mit variablen belegt werden, dass macht das austauschen einfacher.

movie_tile_File = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles.csv'
movie_tile_File_new = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Movie_Titles_clear.gzip'
combined_data_1 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_1.txt'
combined_data_2 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_2.txt'
combined_data_3 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_3.txt'
combined_data_4 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_4.txt'
new_Combined = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Combined_Data_All.gzip'
netflix_rating_Combined = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/netflix_data.csv'
parquet_combined_data = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/data_Comb.zip'

In [ ]:
data = pd.read_parquet(new_Combined)
Convert_dic_data= {'Cust_Id': 'int64', 'Rating': 'float32', 'Movie_Id': 'Int64'}
data= data.astype(Convert_dic_data)
del new_Combined

In [6]:
movie_titles = pd.read_csv(movie_tile_File,
                           encoding = "ISO-8859-1",
                           delimiter= '\t',
                           header = None,
                           names = ['Target'])
                           # Speicher alle Daten erst in eine Reihe, danach trennt er diese
movie_titles[['Movie_Id', 'Year', 'Name']] = movie_titles['Target'].str.split(pat=",",n=2, expand=True)   
movie_titles= movie_titles.drop(['Target', 'Year'], axis= 1)  
Convert_dic= {'Movie_Id': 'int64', 'Name': 'str'}
movie_titles = movie_titles.astype(Convert_dic)
#movie_titles.astype({'Movie_Id': 'int64', 'Name': 'str'}).dtypes
#movie_titles.rename(columns={'Id':'Movie_Id'}, inplace=True)
#movie_titles = movie_titles.reset_index(drop=True)
display(movie_titles)


,Movie_Id,Name
0,1,Dinosaur Planet
1,2,Isle of Man TT 2004 Review
2,3,Character
3,4,Paula Abdul's Get Up & Dance
4,5,The Rise and Fall of ECW
...,...,...
17765,17766,Where the Wild Things Are and Other Maurice Se...
17766,17767,Fidel Castro: American Experience
17767,17768,Epoch
17768,17769,The Company


In [ ]:
num_users = len(data.Cust_Id.unique())
num_items = len(data.Movie_Id.unique())
print('There are {} unique users and {} unique movies in this data set'.format(num_users, num_items))

In [ ]:
df_ratings_cnt_tmp = pd.DataFrame(data.groupby('Rating').size(), columns=['count'])
df_ratings_cnt_tmp

In [ ]:
data = data.reset_index(drop=True)
print(data.head(3))

In [ ]:
# get rating frequency
df_movies_cnt = pd.DataFrame(data.groupby('Movie_Id').size(), columns=['count'])
df_movies_cnt.head()
df_movies_cnt['count'].quantile(np.arange(1, 0.6, -0.05))

In [ ]:
# filter data
popularity_thres = 17700
popular_movies = list(set(df_movies_cnt.query('count >= @popularity_thres').index))
df_ratings_drop_movies = data[data.Movie_Id.isin(popular_movies)]
print('shape of original ratings data: ', data.shape)
print('shape of ratings data after dropping unpopular movies: ', df_ratings_drop_movies.shape)

In [ ]:
# get number of ratings given by every user
df_users_cnt = pd.DataFrame(df_ratings_drop_movies.groupby('Cust_Id').size(), columns=['count'])
df_users_cnt.head()

In [ ]:
df_users_cnt['count'].quantile(np.arange(1, 0.5, -0.05))

In [ ]:
# filter data
ratings_thres = 500
active_users = list(set(df_users_cnt.query('count >= @ratings_thres').index))
df_ratings_drop_users = df_ratings_drop_movies[df_ratings_drop_movies.Cust_Id.isin(active_users)]
print('shape of original ratings data: ', data.shape)
print('shape of ratings data after dropping both unpopular movies and inactive users: ', df_ratings_drop_users.shape)

In [8]:
df = data
# Filter sparse movies
min_movie_ratings = 10000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings


print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

Shape User-Ratings unfiltered:	(100480507, 3)
Shape User-Ratings filtered:	(60546559, 3)


In [12]:
# pivot and create movie-user matrix

movie_user_mat = df_filterd.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)


In [68]:
display(movie_user_mat) 

Cust_Id,6,7,10,79,97,116,134,169,188,195,...,2649308,2649328,2649331,2649335,2649336,2649370,2649378,2649388,2649426,2649429
Movie_Id,,,,,,,,,,,,,,,,,,,,,
8,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0
30,3.0,5.0,0.0,3.0,0.0,4.0,0.0,0.0,3.0,0.0,...,0.0,4.0,5.0,0.0,0.0,0.0,3.0,3.0,4.0,5.0
58,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17697,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17703,2.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
17709,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


In [73]:
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].Name))
}

mapper=movie_to_idx,
match_tuple = []
    # get match
for title, idx in mapper.items():
    ratio = fuzz.ratio(title.lower(), fav_movie.lower())
    if ratio >= 60:
    match_tuple.append((title, idx, ratio))

IndentationError: expected an indented block (178818136.py, line 12)

In [84]:
# We choose random movie.
#query_index = 8585
#query_index = (movie_user_mat.shape[0])
query_index = np.random.choice(movie_user_mat.shape[0])

print("Choosen Movie is: ",movie_user_mat.index[query_index])

print(movie_user_mat.index[query_index])


movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].Name))
}


Choosen Movie is:  4870
4870


In [27]:
user_movie_table_matrix = csr_matrix(movie_user_mat.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(user_movie_table_matrix)
distances, indices = model_knn.kneighbors(movie_user_mat.iloc[query_index,:].values.reshape(1,-1), n_neighbors = 60)

In [85]:
movie = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        movie.append(movie_user_mat.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

m=pd.Series(movie,name='Movie_Id')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

print('Recommendations for {0}:\n'.format(movie_user_mat.index[query_index]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, recommend["Movie_Id"].iloc[i], recommend["distance"].iloc[i]))

Recommendations for 4870:

0: 9828, with distance of 0.5404785871505737
1: 15953, with distance of 0.5397168397903442
2: 7879, with distance of 0.5390525460243225
3: 4661, with distance of 0.5376460552215576
4: 15342, with distance of 0.536626398563385
5: 15182, with distance of 0.5360991954803467
6: 17339, with distance of 0.536009669303894
7: 7521, with distance of 0.5337066650390625
8: 2095, with distance of 0.5330919027328491
9: 12280, with distance of 0.5327714681625366
10: 6337, with distance of 0.5324186682701111
11: 8596, with distance of 0.5316274166107178
12: 14670, with distance of 0.5303036570549011
13: 705, with distance of 0.5297330617904663
14: 9049, with distance of 0.5297234058380127
15: 16438, with distance of 0.529350996017456
16: 17560, with distance of 0.5276150107383728
17: 329, with distance of 0.5271923542022705
18: 6978, with distance of 0.5268598794937134
19: 295, with distance of 0.52669358253479
20: 9593, with distance of 0.5266933441162109
21: 14584, with d

In [ ]:
# pivot and create movie-user matrix
del data, 
movie_user_mat = df_filterd.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)

# create mapper from movie title to index
movie_to_idx = {
   movie: i for i, movie in 
    enumerate(list(movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].Name))
}
# transform matrix to scipy sparse matrix
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [ ]:
# pivot and create movie-user matrix
movie_user_mat = df_ratings_drop_users.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)

# create mapper from movie title to index
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].Name))
}
# transform matrix to scipy sparse matrix
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [ ]:
def fuzzy_matching(mapper, fav_movie, verbose=True):
    #https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]



def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input movie:', fav_movie)
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[idx], dist))

In [ ]:
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=70, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)

In [ ]:
my_favorite = 'Lord of the Rings:'

make_recommendation(
    model_knn=model_knn,
    data=movie_user_mat_sparse,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=5)

In [ ]:
## Läuft nicht!!!
# https://datascience.stackexchange.com/questions/52704/how-to-save-a-knn-model

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

"""
model_knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=70, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)
"""
knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=70, n_jobs=-1)
knn.fit(movie_user_mat_sparse)

# Its important to use binary mode 
knnpickle_file = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/KNN-Model.sav'
knnPickle = open('knnpickle_file', 'wb') 

# source, destination 
pickle.dump(knn, knnPickle)  

# close the file
knnPickle.close()
                
      
# load the model from disk
loaded_model = pickle.load(open('knnpickle_file', 'rb'))
result = loaded_model.predict(movie_user_mat_sparse) 